In [1]:
import gymnasium as gym

In [4]:
env = gym.make('LunarLander-v2', render_mode="human")
observation, info = env.reset()

for _ in range (1000):
    action = env.action_space.sample()
    observation, reward, terminated, trucated, info = env.step(action)
    
    if terminated or trucated:
        observation, info = env.reset()
        
env.close()

In [3]:
class BondGraphEnv(gym.Env):
    metadata = {"render_modes": None, "render_fps": 1}
    
    def __init__(self) -> None:
        super().__init__()
        
    self.observation_space = 
    

SyntaxError: invalid syntax (3977249742.py, line 7)